In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
[MNIST] Classifier Baseline for Starter✨(99.4%)
</h1>
</div>
</div>

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3004/logos/header.png">

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Loading Dataset
</h1>
</div>
</div>

In [ ]:
pd_train = pd.read_csv('../input/digit-recognizer/train.csv')
pd_test = pd.read_csv('../input/digit-recognizer/test.csv')

x_train = pd_train.drop(['label'], axis=1)
y_train = pd_train['label']
x_test = pd_test

In [ ]:
import matplotlib.pyplot as plt

x_train_vis = np.array(x_train).reshape(x_train.shape[0], 28, 28)

fig, axis = plt.subplots(1, 4, figsize=(20, 10))
for i, ax in enumerate(axis.flat):
    ax.imshow(x_train_vis[i], cmap='binary')
    digit = y_train[i]
    ax.set(title = f"Real Number is {digit}");

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Checking Class Imbalance
</h1>
</div>
</div>

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

x = np.unique(y_train)
y = y_train.value_counts().sort_index()
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
            marker_color=x
        )])

fig.update_layout(title_text='Label Count')
fig.show()

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Visualizing 3D t-SNE
</h1>
</div>
</div>

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state = 42, n_components=3, verbose=0, perplexity=40, n_iter=300).fit_transform(x_train)

y_train_label = y_train.astype(str)
fig = px.scatter_3d(pd_train, x=tsne[:,0],
                 y=tsne[:,1],
                 z=tsne[:,2],
                 color=y_train_label)
fig.update_layout(
                margin=dict(l=20, r=20, t=20, b=20)
)
fig.update_traces(marker_size=1.5)
iplot(fig)

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Data Preprocessing
</h1>
</div>
</div>

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
from sklearn.decomposition import PCA

def run_PCA(X, num_components):
    pca = PCA(n_components=num_components)
    pca.fit(X)
    pca_array = pca.transform(X)
    return pca, pca_array

# If you want to use PCA..
# _, x_train = run_PCA(x_train, 3)
# _, x_test = run_PCA(x_test, 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=34)

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Confusion Matrix
</h1>
</div>
</div>

In [ ]:
def plot_confusion_matrix(cm, labels, title):
    # cm : confusion matrix list(list)
    # labels : name of the data list(str)
    # title : title for the heatmap
    data = go.Heatmap(z=cm, y=labels, x=labels)
    annotations = []
    for i, row in enumerate(cm):
        for j, value in enumerate(row):
            annotations.append(
                {
                    "x": labels[i],
                    "y": labels[j],
                    "font": {"color": "white"},
                    "text": str(value),
                    "xref": "x1",
                    "yref": "y1",
                    "showarrow": False
                }
            )
    layout = {
        "title": title,
        "xaxis": {"title": "Predicted value"},
        "yaxis": {"title": "Real value"},
        "annotations": annotations
    }
    fig = go.Figure(data=data, layout=layout)
    return fig

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
KNN Classifier
</h1>
</div>
</div>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

modelKNN = KNeighborsClassifier(n_neighbors=5, p=2)
modelKNN.fit(X_train,Y_train)
y_pred= modelKNN.predict(X_val)
M = confusion_matrix(Y_val, y_pred)
plot_confusion_matrix(M, sorted(Y_val.unique()), 'KNN Confusion Matrix, Acc({:.2f}%)'.format(accuracy_score(Y_val, y_pred)*100))

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Decision Tree Classifier
</h1>
</div>
</div>

In [ ]:
modelDT = DecisionTreeClassifier()
modelDT.fit(X_train,Y_train)
y_pred= modelDT.predict(X_val)
M = confusion_matrix(Y_val, y_pred)
plot_confusion_matrix(M, sorted(Y_val.unique()), 'Decision Tree Confusion Matrix, Acc({:.2f}%)'.format(accuracy_score(Y_val, y_pred)*100))

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Random Forest Classifier
</h1>
</div>
</div>

In [ ]:
modelRF = RandomForestClassifier()
modelRF.fit(X_train,Y_train)
y_pred= modelRF.predict(X_val)
M = confusion_matrix(Y_val, y_pred)
plot_confusion_matrix(M, sorted(Y_val.unique()), 'Random Forest Confusion Matrix, Acc({:.2f}%)'.format(accuracy_score(Y_val, y_pred)*100))

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Light CNN Classifier
</h1>
</div>
</div>

<img src="https://taewanmerepo.github.io/2018/01/cnn/head.png">

### CNN is a very efficient ways for computer vision tasks. 
a convolutional neural network (CNN, or ConvNet) is a class of Artificial Neural Network (ANN), most commonly applied to analyze visual imagery. They are also known as Shift Invariant or Space Invariant Artificial Neural Networks (SIANN), based on the shared-weight architecture of the convolution kernels or filters that slide along input features and provide translation equivariant responses known as feature maps. Counter-intuitively, most convolutional neural networks are only equivariant, as opposed to invariant, to translation. They have applications in image and video recognition, recommender systems, image classification, image segmentation, medical image analysis, natural language processing, brain-computer interfaces, and financial time series.

In [ ]:
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import datasets, layers, models

In [ ]:
X_train = np.array(X_train).reshape(-1,28,28,1)
X_val = np.array(X_val).reshape(-1,28,28,1)
Y_train = np_utils.to_categorical(np.array(Y_train))
Y_val = np_utils.to_categorical(np.array(Y_val))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation = "softmax"))

model.summary()

In [ ]:
model_path = './best_model.h5'
checkpointer = ModelCheckpoint(
    filepath=model_path,
    monitor='val_accuracy',
    verbose=1,
    mode='max',
    save_best_only=True)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs=50, batch_size = 128, callbacks=[checkpointer])

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

fig = go.Figure()

labels = ['Loss', 'Accuracy']
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(1, 51),)*2)

y_data = np.array([
    np.array(history.history['loss']), 
                      np.array(history.history['accuracy'])
])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {:.2f}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[49],
                                  xanchor='left', yanchor='middle',
                                  text='{:.4f}%'.format(y_trace[49]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Training Loss & Accuracy',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Epochs',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,height=600, width=1000)

fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

fig = go.Figure()

labels = ['Val Loss', 'Val Accuracy']
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(1, 51),)*2)

y_data = np.array([
    np.array(history.history['val_loss']), 
                      np.array(history.history['val_accuracy'])
])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {:.2f}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[49],
                                  xanchor='left', yanchor='middle',
                                  text='{:.4f}%'.format(y_trace[49]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Validation Loss & Accuracy',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Epochs',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,height=600, width=1000)

fig.show()


In [ ]:
from keras.models import load_model
model = load_model('./best_model.h5')
y_pred = model.predict(X_val)
M = confusion_matrix(np.argmax(Y_val,1), np.argmax(y_pred,1))
plot_confusion_matrix(M, sorted(np.unique(np.argmax(Y_val,1))), 'Simple CNN Confusion Matrix, Acc({:.2f}%)'.format(accuracy_score(np.argmax(Y_val,1), np.argmax(y_pred,1))*100))

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Checking Failure Cases
</h1>
</div>
</div>

In [ ]:
n = np.where(np.argmax(y_pred,1)!=np.argmax(Y_val,1))
n = np.array(n).reshape(-1)
n

In [ ]:
X_val_vis = np.array(X_val).reshape(X_val.shape[0], 28, 28)
X_val_vis = X_val_vis*255
fig, axis = plt.subplots(int(len(n)/8), 8, figsize=(35, 15))
for i, ax in enumerate(axis.flat):
    ax.imshow(X_val_vis[n[i]], cmap='binary')
    digit = np.argmax(y_pred,1)[n[i]]
    digit2 = np.argmax(Y_val,1)[n[i]]
    ax.set(title = f"Pred Number is {digit}\n Real Number is {digit2}");

### It looks like even human can't get the right answers.😅

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:gray;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Submit to Leaderboard
</h1>
</div>
</div>

In [ ]:
X_test = np.array(x_test).reshape(-1,28,28,1)

In [ ]:
model = load_model('./best_model.h5')
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,1)
pd_submit = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
pd_submit['Label'] = y_pred
pd_submit.to_csv('dub.csv', header=True, index=False)

## Consider upvoting if it was helpful! 😃